In [1]:
import itertools
import os

import pandas as pd
from scipy.stats import mannwhitneyu
import numpy as np

import utils

In [2]:
# Input parameters
country_codes = ['ESP', 'USA', 'EST', 'ETH']
dem_names = ['AW3D30', 'COP30', 'HydroSHEDS', 'MERIT', 'NASADEM', 'TanDEM']
feature_types = ['basin', 'stream']
stat_names = ['forest_pct', 'slope_median']

In [3]:
for stat_name in stat_names:
    for feature_type in feature_types:
        out_fp = f'D:/dem_comparison/data/mannwhitneyu_{stat_name}_{feature_type}.xlsx'
        if os.path.exists(out_fp):
            os.remove(out_fp)
        for country_code in country_codes:
            merged = utils.merge_stats_for_plot(country_code, dem_names, feature_type, stat_name)
            merged[f'{stat_name}_class'] = merged.apply(utils.get_class_func(stat_name), axis=1)
            stat_classes = merged[f'{stat_name}_class'].unique()
            row_list = []
            for dem_name in dem_names:
                for class_pair in itertools.combinations(stat_classes, 2):
                    class_1 = class_pair[0]
                    class_2 = class_pair[1]
                    subset_1 = merged.loc[merged[f'{stat_name}_class'] == class_1]
                    subset_2 = merged.loc[merged[f'{stat_name}_class'] == class_2]
                    x = subset_1.loc[subset_1['dem_name'] == dem_name]['dist_to_ref'].to_list()
                    y = subset_2.loc[subset_2['dem_name'] == dem_name]['dist_to_ref'].to_list()
                    try:
                        U1, p = mannwhitneyu(x, y)
                        U1 = round(U1, 1)
                        p = round(p, 3)
                    except ValueError:
                        U1 = np.nan
                        p = np.nan
                    catchment_name = utils.get_catchment_name(country_code)
                    if p < 0.05:
                        significance = 1
                    else:
                        significance = 0
                    row = (
                        catchment_name,
                        country_code,
                        dem_name,
                        feature_type,
                        stat_name,
                        class_1,
                        len(x),
                        class_2,
                        len(y),
                        U1,
                        p,
                        significance
                    )
                    row_list.append(row)
            out_df_columns = [
                'catchment_name',
                'country_code',
                'dem_name',
                'feature_type',
                'stat_name',
                'class_1',
                'count_1',
                'class_2',
                'count_2',
                'U',
                'p',
                'significant'
            ]
            out_df = pd.DataFrame(row_list, columns=out_df_columns)
            display(out_df)
            if not os.path.exists(out_fp):
                out_df.to_excel(out_fp, sheet_name=catchment_name, index=False)
            else:
                with pd.ExcelWriter(out_fp, mode='a') as writer:
                    out_df.to_excel(writer, sheet_name=catchment_name, index=False)

,catchment_name,country_code,dem_name,feature_type,stat_name,class_1,count_1,class_2,count_2,U,p,significant
0,Argos,ESP,AW3D30,basin,forest_pct,< 10,438,10 - 25,117,29589.5,0.010,1
1,Argos,ESP,AW3D30,basin,forest_pct,< 10,438,25 - 50,263,65909.5,0.001,1
2,Argos,ESP,AW3D30,basin,forest_pct,< 10,438,> 50,1006,257078.0,0.000,1
3,Argos,ESP,AW3D30,basin,forest_pct,10 - 25,117,25 - 50,263,15350.0,0.972,0
4,Argos,ESP,AW3D30,basin,forest_pct,10 - 25,117,> 50,1006,60301.0,0.662,0
5,Argos,ESP,AW3D30,basin,forest_pct,25 - 50,263,> 50,1006,138296.0,0.256,0
6,Argos,ESP,COP30,basin,forest_pct,< 10,442,10 - 25,111,26142.0,0.285,0
7,Argos,ESP,COP30,basin,forest_pct,< 10,442,25 - 50,242,58480.0,0.043,1
8,Argos,ESP,COP30,basin,forest_pct,< 10,442,> 50,986,244489.0,0.000,1
9,Argos,ESP,COP30,basin,forest_pct,10 - 25,111,25 - 50,242,13925.0,0.579,0


,catchment_name,country_code,dem_name,feature_type,stat_name,class_1,count_1,class_2,count_2,U,p,significant
0,Bald Eagle,USA,AW3D30,basin,forest_pct,< 10,289,10 - 25,81,11474.0,0.787,0
1,Bald Eagle,USA,AW3D30,basin,forest_pct,< 10,289,25 - 50,110,17802.0,0.064,0
2,Bald Eagle,USA,AW3D30,basin,forest_pct,< 10,289,> 50,3006,433480.0,0.954,0
3,Bald Eagle,USA,AW3D30,basin,forest_pct,10 - 25,81,25 - 50,110,5108.0,0.084,0
4,Bald Eagle,USA,AW3D30,basin,forest_pct,10 - 25,81,> 50,3006,124140.0,0.762,0
5,Bald Eagle,USA,AW3D30,basin,forest_pct,25 - 50,110,> 50,3006,144929.0,0.028,1
6,Bald Eagle,USA,COP30,basin,forest_pct,< 10,289,10 - 25,69,9126.0,0.275,0
7,Bald Eagle,USA,COP30,basin,forest_pct,< 10,289,25 - 50,111,16238.0,0.848,0
8,Bald Eagle,USA,COP30,basin,forest_pct,< 10,289,> 50,3002,448453.0,0.342,0
9,Bald Eagle,USA,COP30,basin,forest_pct,10 - 25,69,25 - 50,111,4168.0,0.320,0


,catchment_name,country_code,dem_name,feature_type,stat_name,class_1,count_1,class_2,count_2,U,p,significant
0,Porijõgi,EST,AW3D30,basin,forest_pct,< 10,142,10 - 25,57,3377.0,0.068,0
1,Porijõgi,EST,AW3D30,basin,forest_pct,< 10,142,25 - 50,97,5056.0,0.000,1
2,Porijõgi,EST,AW3D30,basin,forest_pct,< 10,142,> 50,959,49679.0,0.000,1
3,Porijõgi,EST,AW3D30,basin,forest_pct,10 - 25,57,25 - 50,97,2522.0,0.365,0
4,Porijõgi,EST,AW3D30,basin,forest_pct,10 - 25,57,> 50,959,25576.0,0.415,0
5,Porijõgi,EST,AW3D30,basin,forest_pct,25 - 50,97,> 50,959,48330.0,0.525,0
6,Porijõgi,EST,COP30,basin,forest_pct,< 10,203,10 - 25,81,8897.0,0.280,0
7,Porijõgi,EST,COP30,basin,forest_pct,< 10,203,25 - 50,103,9951.0,0.492,0
8,Porijõgi,EST,COP30,basin,forest_pct,< 10,203,> 50,985,82685.0,0.000,1
9,Porijõgi,EST,COP30,basin,forest_pct,10 - 25,81,25 - 50,103,3536.0,0.077,0


,catchment_name,country_code,dem_name,feature_type,stat_name,class_1,count_1,class_2,count_2,U,p,significant
0,Rib,ETH,AW3D30,basin,forest_pct,< 10,2351,10 - 25,242,280828.0,0.743,0
1,Rib,ETH,AW3D30,basin,forest_pct,< 10,2351,25 - 50,191,231733.0,0.460,0
2,Rib,ETH,AW3D30,basin,forest_pct,< 10,2351,> 50,122,162886.0,0.011,1
3,Rib,ETH,AW3D30,basin,forest_pct,10 - 25,242,25 - 50,191,24167.0,0.414,0
4,Rib,ETH,AW3D30,basin,forest_pct,10 - 25,242,> 50,122,17000.0,0.018,1
5,Rib,ETH,AW3D30,basin,forest_pct,25 - 50,191,> 50,122,12985.0,0.088,0
6,Rib,ETH,COP30,basin,forest_pct,< 10,2345,10 - 25,236,265313.0,0.296,0
7,Rib,ETH,COP30,basin,forest_pct,< 10,2345,25 - 50,187,214400.0,0.614,0
8,Rib,ETH,COP30,basin,forest_pct,< 10,2345,> 50,127,174767.0,0.001,1
9,Rib,ETH,COP30,basin,forest_pct,10 - 25,236,25 - 50,187,22523.0,0.715,0


,catchment_name,country_code,dem_name,feature_type,stat_name,class_1,count_1,class_2,count_2,U,p,significant
0,Argos,ESP,AW3D30,stream,forest_pct,< 10,1122,10 - 25,411,276869.5,0.000,1
1,Argos,ESP,AW3D30,stream,forest_pct,< 10,1122,25 - 50,640,462161.5,0.000,1
2,Argos,ESP,AW3D30,stream,forest_pct,< 10,1122,> 50,1247,977672.0,0.000,1
3,Argos,ESP,AW3D30,stream,forest_pct,10 - 25,411,25 - 50,640,141959.5,0.030,1
4,Argos,ESP,AW3D30,stream,forest_pct,10 - 25,411,> 50,1247,305238.0,0.000,1
5,Argos,ESP,AW3D30,stream,forest_pct,25 - 50,640,> 50,1247,444724.0,0.000,1
6,Argos,ESP,COP30,stream,forest_pct,< 10,935,10 - 25,344,177242.5,0.005,1
7,Argos,ESP,COP30,stream,forest_pct,< 10,935,25 - 50,537,303411.5,0.000,1
8,Argos,ESP,COP30,stream,forest_pct,< 10,935,> 50,1008,609139.0,0.000,1
9,Argos,ESP,COP30,stream,forest_pct,10 - 25,344,25 - 50,537,104248.5,0.001,1


...

,catchment_name,country_code,dem_name,feature_type,stat_name,class_1,count_1,class_2,count_2,U,p,significant
0,Rib,ETH,AW3D30,basin,slope_median,< 5,1104,5 - 10,764,489926.0,0.000,1
1,Rib,ETH,AW3D30,basin,slope_median,< 5,1104,10 - 40,1036,690148.0,0.000,1
2,Rib,ETH,AW3D30,basin,slope_median,< 5,1104,> 40,2,934.0,0.717,0
3,Rib,ETH,AW3D30,basin,slope_median,5 - 10,764,10 - 40,1036,417541.0,0.046,1
4,Rib,ETH,AW3D30,basin,slope_median,5 - 10,764,> 40,2,478.0,0.393,0
5,Rib,ETH,AW3D30,basin,slope_median,10 - 40,1036,> 40,2,573.0,0.307,0
6,Rib,ETH,COP30,basin,slope_median,< 5,1071,5 - 10,772,455530.0,0.000,1
7,Rib,ETH,COP30,basin,slope_median,< 5,1071,10 - 40,1051,634510.0,0.000,1
8,Rib,ETH,COP30,basin,slope_median,< 5,1071,> 40,1,1071.0,0.002,1
9,Rib,ETH,COP30,basin,slope_median,5 - 10,772,10 - 40,1051,417979.0,0.268,0


,catchment_name,country_code,dem_name,feature_type,stat_name,class_1,count_1,class_2,count_2,U,p,significant
0,Argos,ESP,AW3D30,stream,slope_median,< 5,1865,5 - 10,938,1158159.0,0.000,1
1,Argos,ESP,AW3D30,stream,slope_median,< 5,1865,10 - 40,617,826935.5,0.000,1
2,Argos,ESP,AW3D30,stream,slope_median,5 - 10,938,10 - 40,617,324789.5,0.000,1
3,Argos,ESP,COP30,stream,slope_median,< 5,1546,5 - 10,790,728308.0,0.000,1
4,Argos,ESP,COP30,stream,slope_median,< 5,1546,10 - 40,488,464572.5,0.000,1
5,Argos,ESP,COP30,stream,slope_median,5 - 10,790,10 - 40,488,201098.5,0.193,0
6,Argos,ESP,HydroSHEDS,stream,slope_median,< 5,1609,5 - 10,805,699371.5,0.001,1
7,Argos,ESP,HydroSHEDS,stream,slope_median,< 5,1609,10 - 40,696,573751.5,0.346,0
8,Argos,ESP,HydroSHEDS,stream,slope_median,5 - 10,805,10 - 40,696,261188.5,0.024,1
9,Argos,ESP,MERIT,stream,slope_median,< 5,1683,5 - 10,859,902241.5,0.000,1


,catchment_name,country_code,dem_name,feature_type,stat_name,class_1,count_1,class_2,count_2,U,p,significant
0,Bald Eagle,USA,AW3D30,stream,slope_median,< 5,3436,5 - 10,3288,7018770.0,0.000,1
1,Bald Eagle,USA,AW3D30,stream,slope_median,< 5,3436,10 - 40,3487,8741743.0,0.000,1
2,Bald Eagle,USA,AW3D30,stream,slope_median,5 - 10,3288,10 - 40,3487,7199381.5,0.000,1
3,Bald Eagle,USA,COP30,stream,slope_median,< 5,3059,5 - 10,3017,5748937.0,0.000,1
4,Bald Eagle,USA,COP30,stream,slope_median,< 5,3059,10 - 40,3060,6755869.0,0.000,1
5,Bald Eagle,USA,COP30,stream,slope_median,5 - 10,3017,10 - 40,3060,5766129.5,0.000,1
6,Bald Eagle,USA,HydroSHEDS,stream,slope_median,< 5,2945,5 - 10,3025,5132731.5,0.000,1
7,Bald Eagle,USA,HydroSHEDS,stream,slope_median,< 5,2945,10 - 40,3456,6092573.5,0.000,1
8,Bald Eagle,USA,HydroSHEDS,stream,slope_median,5 - 10,3025,10 - 40,3456,5487261.0,0.001,1
9,Bald Eagle,USA,MERIT,stream,slope_median,< 5,2732,5 - 10,3287,5344193.0,0.000,1


,catchment_name,country_code,dem_name,feature_type,stat_name,class_1,count_1,class_2,count_2,U,p,significant
0,Porijõgi,EST,AW3D30,stream,slope_median,< 5,1481,5 - 10,238,172739.0,0.622,0
1,Porijõgi,EST,AW3D30,stream,slope_median,< 5,1481,10 - 40,33,25673.0,0.619,0
2,Porijõgi,EST,AW3D30,stream,slope_median,5 - 10,238,10 - 40,33,4161.0,0.580,0
3,Porijõgi,EST,COP30,stream,slope_median,< 5,1115,5 - 10,260,142236.0,0.638,0
4,Porijõgi,EST,COP30,stream,slope_median,< 5,1115,10 - 40,42,23954.0,0.800,0
5,Porijõgi,EST,COP30,stream,slope_median,5 - 10,260,10 - 40,42,5667.0,0.694,0
6,Porijõgi,EST,HydroSHEDS,stream,slope_median,< 5,1190,5 - 10,236,159490.0,0.001,1
7,Porijõgi,EST,HydroSHEDS,stream,slope_median,< 5,1190,10 - 40,23,14554.0,0.602,0
8,Porijõgi,EST,HydroSHEDS,stream,slope_median,5 - 10,236,10 - 40,23,2467.0,0.472,0
9,Porijõgi,EST,MERIT,stream,slope_median,< 5,1155,5 - 10,392,220994.0,0.481,0


,catchment_name,country_code,dem_name,feature_type,stat_name,class_1,count_1,class_2,count_2,U,p,significant
0,Rib,ETH,AW3D30,stream,slope_median,< 5,4947,5 - 10,2964,8892262.5,0.000,1
1,Rib,ETH,AW3D30,stream,slope_median,< 5,4947,10 - 40,2286,7117902.5,0.000,1
2,Rib,ETH,AW3D30,stream,slope_median,< 5,4947,> 40,7,20202.0,0.445,0
3,Rib,ETH,AW3D30,stream,slope_median,5 - 10,2964,10 - 40,2286,3563507.5,0.001,1
4,Rib,ETH,AW3D30,stream,slope_median,5 - 10,2964,> 40,7,9972.0,0.859,0
5,Rib,ETH,AW3D30,stream,slope_median,10 - 40,2286,> 40,7,7365.5,0.717,0
6,Rib,ETH,COP30,stream,slope_median,< 5,4209,5 - 10,2451,6162970.0,0.000,1
7,Rib,ETH,COP30,stream,slope_median,< 5,4209,10 - 40,1883,4938789.5,0.000,1
8,Rib,ETH,COP30,stream,slope_median,< 5,4209,> 40,6,19049.0,0.031,1
9,Rib,ETH,COP30,stream,slope_median,5 - 10,2451,10 - 40,1883,2430843.5,0.003,1
